In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)

In [3]:
df=pd.read_csv(r'/Users/rooblebagga/Documents/2020-12-14-AccountStatement.csv')

In [4]:
#Cash Balance Data Set 
x=df[df["Account Statement"]=="Cash Balance"].index.values
y=df[df["Account Statement"]=="Futures Statements"].index.values
Cash_Balance = df[x[0]+1:y[0]-1]
Cash_Balance=Cash_Balance.rename(columns=Cash_Balance.iloc[0]).drop(Cash_Balance.index[0]).reset_index()
Cash_Balance = Cash_Balance.loc[:, Cash_Balance.columns.notnull()]
Cash_Balance.drop(['index'],axis=1, inplace=True)

In [5]:
a=df[df["Account Statement"]=="Account Order History"].index.values
b=df[df["Account Statement"]=="Account Trade History"].index.values
Order_History = df[a[0]+1:b[0]-1]

In [6]:
c=df[df["Account Statement"]=="Equities"].index.values
d=df[df["Account Statement"]=="Profits and Losses"].index.values
Equities = df[c[0]+1:d[0]-2]
Equities=Equities.rename(columns=Equities.iloc[0]).drop(Equities.index[0]).reset_index()
Equities = Equities.loc[:, Equities.columns.notnull()]
Equities.drop(['index'],axis=1, inplace=True)

In [7]:
e=df[df["Account Statement"]=="Profits and Losses"].index.values
f=df[df["Account Statement"]=="Account Summary"].index.values
PnL = df[e[0]+1:f[0]-2]
PnL=PnL.rename(columns=PnL.iloc[0]).drop(PnL.index[0]).reset_index()
PnL = PnL.loc[:, PnL.columns.notnull()]
PnL.drop(['index'],axis=1, inplace=True)

In [8]:
Order_History=Order_History.rename(columns=Order_History.iloc[0]).drop(Order_History.index[0]).reset_index()
Order_History.drop(['Notes', 'T','index'],axis=1, inplace=True)

In [9]:
Order_History = Order_History.rename(columns={np.nan:'Order Type', 'PRICE':'Price'})
Order_History['Price']=Order_History['Price'].replace({'~':np.nan})
Order_History.drop(['Exp', 'Strike','Type'],axis=1, inplace=True)

In [210]:
# def applyFunc(s):
#     if s.str.contains('%'):
#         return 'Yes'
#     else:
#         return 'No'
# Order_History['Qty %'] = Order_History['Qty'].apply(applyFunc)

In [12]:
Equities

,Symbol,Description,Qty,Trade Price,Mark,Mark Value
0,SPCE,VIRGIN GALACTIC HLDGS INC COM,200,34.6026,26.51,"$5,302.00"
1,AAL,AMERICAN AIRLINES GROUP INC COM,200,18.2498,16.9861,"$3,397.22"


In [13]:
PnL =PnL.replace({'\$':'', '\^':'','%':'',',':''}, regex=True)
PnL['P/L Open'] = PnL['P/L Open'].astype(str).str.replace('\((.*)\)', '-\\1')
PnL['P/L %'] = PnL['P/L %'].astype(str).str.replace('\((.*)\)', '-\\1')
PnL['P/L Day'] = PnL['P/L Day'].astype(str).str.replace('\((.*)\)', '-\\1')
PnL['P/L YTD'] = PnL['P/L YTD'].astype(str).str.replace('\((.*)\)', '-\\1')
PnL['P/L Diff'] = PnL['P/L Diff'].astype(str).str.replace('\((.*)\)', '-\\1')

PnL['P/L Open']=PnL['P/L Open'].astype(float)
PnL['P/L %']=PnL['P/L %'].astype(float)
PnL['P/L Day']=PnL['P/L Day'].astype(float)
PnL['P/L YTD']=PnL['P/L YTD'].astype(float)
PnL['P/L Diff']=PnL['P/L Diff'].astype(float)
PnL['Margin Req']=PnL['Margin Req'].astype(float)
PnL['Mark Value']=PnL['Mark Value'].astype(float)


In [14]:
df2=Cash_Balance.copy() 

In [15]:
trades=df2[df2['TYPE']=='TRD']
description=trades['DESCRIPTION'].str.split(expand = True)
description = description.rename(columns={0:'Order Type', 1:'Quantity',2:'Stock',3:'Price'})
description =description.replace({'@':''}, regex=True)
description['Price']=description['Price'].astype(float)
description['Quantity']=description['Quantity'].astype(float)
df_merged = trades.merge(description, how='outer', left_index=True, right_index=True)


In [20]:
df1=df_merged.groupby(["DATE", "Stock"])["Quantity"].count().to_frame(name = 'count')
# df1.groupby( [ "Name", "City"] ).size().to_frame(name = 'count').reset_index()

In [24]:
df1

count
DATE     Stock       
11/12/20 AMD        2
         FB         4
         IMAC       8
         NIO        4
         PTON       4
11/16/20 NIO        3
11/18/20 PFE        2
11/19/20 NIO        2
         PLTR       2
         TSLA       2
11/20/20 BLNK       3
         PLTR       1
11/23/20 BLNK       9
         KXIN       5
         LI         3
         PLTR       1
         PTON       2
         ROKU       5
         XPEV       5
11/24/20 CCL        2
         FTEK       2
         LI         7
         NKLA       7
         OXY        3
         UA         3
11/25/20 NIO        2
         PFE        2
         PLTR       2
11/27/20 AAL        2
         FSLY       3
         INO        4
         NFLX       5
         SAVE       5
         WORK       2
11/30/20 RIOT       2
12/1/20  GME        2
         NNDM       5
         RIOT       2
         TSLA       2
12/10/20 FSLY       2
         NIO        2
         PLTR       2
12/11/20 NIO       10
         SNOW      24
         TSLA       2
12/14/20 AAL        1
         LULU       2
         PLTR       2
         SNOW       2
12/2/20  BB         2
         GME        2
         INO        3
         NIO        7
         SPOT       7
         TSLA       8
         WKHS       3
12/3/20  NIO        5
12/4/20  FSLY       6
         GME        6
         NIO        2
         SNOW       3
         TSLA       5
12/7/20  NIO        4
         PLTR       7
12/8/20  ANF       11
         ETSY       1
         FSLY      10
         INO        3
         MRNA       4
         NIO        5
         NKLA       4
         SNOW       9
         TSLA       6
12/9/20  AAL        1
         ANF        1
         ETSY       1
         SPCE       1

In [23]:
df1.columns

Index(['count'], dtype='object')

In [10]:
Order_History

,Time Placed,Spread,Side,Qty,Pos Effect,Symbol,Price,Order Type,TIF,Status
0,12/14/20 9:58,STOCK,SELL,-300,TO CLOSE,AAL,NaN,MKT,DAY,FILLED
1,12/14/20 9:50,STOCK,SELL,-20,TO CLOSE,LULU,NaN,MKT,DAY,FILLED
2,12/14/20 9:48,STOCK,BUY,20,TO OPEN,LULU,NaN,MKT,DAY,FILLED
3,12/14/20 9:42,STOCK,SELL,-200,TO CLOSE,PLTR,NaN,MKT,DAY,FILLED
4,12/14/20 9:42,STOCK,SELL,-200,TO CLOSE,PLTR,28.12,LMT,DAY,CANCELED
5,12/14/20 9:40,STOCK,BUY,200,TO OPEN,PLTR,NaN,MKT,DAY,FILLED
6,12/14/20 9:38,STOCK,BUY,22,TO CLOSE,SNOW,342.1,LMT,DAY,FILLED
7,12/14/20 9:36,STOCK,SELL,-22,TO OPEN,SNOW,NaN,MKT,DAY,FILLED
8,12/11/20 11:26,STOCK,SELL,-10,TO CLOSE,SNOW,351.85,LMT,DAY,FILLED
9,12/11/20 11:25,STOCK,BUY,10,TO OPEN,SNOW,NaN,MKT,DAY,FILLED


In [41]:
Order_History.groupby(["Time Placed", "Symbol", "Status","Order Type"])["Qty"].sum()

Time Placed     Symbol  Status          Order Type
11/12/20 10:47  AMD     FILLED          LMT           -20.0
11/12/20 10:49  AMD     CANCELED        LMT            20.0
11/12/20 10:50  AMD     FILLED          MKT            20.0
11/12/20 12:28  BA      CANCELED        LMT           -10.0
11/12/20 13:00  FB      FILLED          LMT            30.0
11/12/20 14:01  PTON    FILLED          LMT          -100.0
11/12/20 14:06  FB      FILLED          MKT           -30.0
11/12/20 14:13  PTON    FILLED          LMT           100.0
11/12/20 14:17  NIO     FILLED          MKT           -50.0
11/12/20 14:18  NIO     FILLED          MKT            50.0
11/12/20 9:33   IMAC    FILLED          LMT           500.0
11/12/20 9:37   IMAC    FILLED          LMT          -500.0
11/12/20 9:42   IMAC    FILLED          LMT           500.0
11/12/20 9:43   IMAC    CANCELED        LMT          -500.0
11/12/20 9:44   IMAC    FILLED          LMT          -500.0
11/12/20 9:46   IMAC    FILLED          LMT      

In [26]:
Order_History =Order_History.replace({'\$':'', '\^':'','%':'',',':''}, regex=True)


In [29]:
Order_History['Qty']=Order_History['Qty'].astype(float)

In [33]:
# Total Volume All Time 
Order_History.assign(AbsScore=Order_History.Qty.abs()).AbsScore.sum()

46137.0

In [34]:
#Market Orders
Market_Orders=Order_History['Order Type'].str.count("MKT").sum()

168.0

In [35]:
#Limit Orders
Limit_Orders=Order_History['Order Type'].str.count("LMT").sum()

128.0

In [36]:
#Number of Cancelled vs Filled
Filled_Orders=Order_History['Status'].str.count("FILLED").sum()

245.0

In [37]:
Canceled_Orders=Order_History['Status'].str.count("CANCELED").sum()

50.0

In [ ]:
df_merged.groupby(["DATE", "Stock"])["Quantity"].count()

In [43]:
df_merged

,DATE,TIME,TYPE,REF #,DESCRIPTION,Misc Fees,Commissions & Fees,AMOUNT,BALANCE,Order Type,Quantity,Stock,Price
29,11/12/20,9:33:59,TRD,3551442842,BOT +500 IMAC @2.16,NaN,NaN,"-1,080.00","22,229.00",BOT,500.0,IMAC,2.16000
30,11/12/20,9:38:11,TRD,3551552718,SOLD -139 IMAC @2.39,-0.03,NaN,332.21,"22,561.18",SOLD,-139.0,IMAC,2.39000
31,11/12/20,9:38:11,TRD,3551552718,SOLD -361 IMAC @2.39,-0.06,NaN,862.79,"23,423.91",SOLD,-361.0,IMAC,2.39000
32,11/12/20,9:42:36,TRD,3551676955,BOT +500 IMAC @2.2288,NaN,NaN,"-1,114.40","22,309.51",BOT,500.0,IMAC,2.22880
33,11/12/20,9:46:13,TRD,3551729657,SOLD -30 IMAC @2.22,-0.01,NaN,66.6,"22,376.10",SOLD,-30.0,IMAC,2.22000
34,11/12/20,9:46:13,TRD,3551729657,SOLD -470 IMAC @2.22,-0.08,NaN,"1,043.40","23,419.42",SOLD,-470.0,IMAC,2.22000
35,11/12/20,9:46:45,TRD,3551769400,BOT +500 IMAC @2.25,NaN,NaN,"-1,125.00","22,294.42",BOT,500.0,IMAC,2.25000
36,11/12/20,9:50:57,TRD,3551864675,SOLD -500 IMAC @1.9701,-0.08,NaN,985.05,"23,279.39",SOLD,-500.0,IMAC,1.97010
37,11/12/20,9:53:36,TRD,3551920670,SOLD -1 NIO @44.90,-0.01,NaN,44.9,"23,324.28",SOLD,-1.0,NIO,44.90000
38,11/12/20,9:53:36,TRD,3551920670,SOLD -49 NIO @44.90,-0.06,NaN,"2,200.10","25,524.32",SOLD,-49.0,NIO,44.90000


In [ ]:
def categorizeDay() {
    if endingBalance > staringBalance:
        return('winning') 
    else:
        return('losing')
}


def numberOfTrades(){
    typeOfDay = categorizeDay()
    if typeOfDay == 'winning '
}

def averageVolume(date){
    quantity = 0
    for i in Order_History:
        if Order_History[i][Time] == date:
            quantity += Order_History[i][Qty]
    return quantity
}

def numOfCanceledOrders(){
    numOfCanceledOrders = 0
    for i in Order_History:
        if Order_History[i][Status] == 'Canceled':
    numOfCanceledOrders += 1
}

def quantifyOrderTypes() {
    totalMarketOrders = 0
    totalLimitOrders = 0
    for i in Order_History:
        if Order_History[i][Type] == 'MKT':
            totalMarketOrders += 1
        else: 
            totalLimitOrders += 1
}